# NCPT Paperspace One-click (v0.3)

### Last updated: 25 Dec 2023

Check for updates [here](https://github.com/NoCrypt/paperspace-ncpt)

**!! ONLY RUN IN JUPYTERLAB !!**

![](https://i.ibb.co/M9ZH6qf/image.png)




To hide the code, press on this blue bar on the left of the cell

![](https://i.ibb.co/gFqKVV6/chrome-23-12-25-195649.png)

---


#### 1. Configurations

In [ ]:
from ipywidgets import Checkbox, Text, Button, HBox, ToggleButton, Label, Box, Layout
from IPython.display import display, HTML
from pathlib import Path
from subprocess import check_output, check_call
import importlib
import json
import os
import requests
from urllib.parse import urlparse, parse_qs, unquote


style = {"description_width": "25%"}
layout = {"width": "70%"}

label_style = HTML(
    "<style> .widget-label, .widget-label-basic >  span { color: white !important; font-weight: bold;}  </style>"
)

persistent_config_control = Checkbox(
    value=False,  # Keep it false as default, presistent is good but I might change config in the future, also it's much more idiot-proof this way
    description="Persistent config",
    layout=layout,
    style=style,
)

persistence_control = Checkbox(
    value=False,  # Keep it false as default, I just prefer it off by default like in old colab
    description="Save outputs to persistant notebook",
    layout=layout,
    style=style,
)

force_cache_control = Checkbox(
    value=False,  # Keep it false as default, cuz why?!
    description="Force use of cached dependencies (faster, may break things)",
    layout=layout,
    style=style,
)

extn_update_control = Checkbox(
    value=True, # You would want newer extensions by default
    description="Update all extensions on launch",
    layout=layout,
    style=style,
)

# textboxes
hf_token = Text(
    value="",
    description="HuggingFace token (optional)",
    placeholder="hf_xyz",
    style=style,
    layout=layout,
)
commandline_arg_control = Text(
    value="--xformers",
    description="Extra commandline arguments",
    style=style,
    layout=layout,
)


output_control = Text(
    value="./outputs/",
    description="Outputs path",
    placeholder="hf_xyz",
    style=style,
    layout=layout,
)

zip_button = Button(
    description='Zip',
    tooltip='Zip folder',
    icon='zipper' 
)
clean_button = Button(
    description='Clean',
    tooltip='Cleaning folder',
    icon='zipper'
)
zip_clean_button = Button(
    description='Zip & Clean',
    button_style='info', 
    tooltip='Zip folder and clean it up',
    icon='file' 
)


# funcs 
def zip_outputs(b):
    display("Cleaning..a.")
    b.description = b.description.split('>')[0]
    is_cleaning = "clean" in b.description.lower()
    is_zipping = "zip" in b.description.lower()
    cleanup_dir = Path(output_control.value)
    if not cleanup_dir.exists():
        display("The folder doesn't exist.")
        #b.style.button_color = 'red'
        b.button_style = 'danger' 
        b.description = b.description+ '> not found...' 
        
    if any(cleanup_dir.iterdir()) :
        if is_zipping: 
            zipped_name = f"{cleanup_dir.name}_{time.strftime('%Y-%m-%d_%H%M')}.zip" 
            !zip -r {zipped_name} {cleanup_dir}
            display(zipped_name+" created")
        if is_cleaning :
            folder_name=cleanup_dir.name
            clean_outputs(cleanup_dir)
            display(f"{folder_name} cleaned!")
        #b.style.button_color = 'green'
        b.button_style = 'success' 
        b.description =b.description+ '> done!' 
        
    else :
        display("Empty folder.")
        #b.style.button_color = 'orange'
        b.button_style = 'warning' 
        b.description =b.description + '>empty..' 
    
def clean_outputs(b):
    !rm -rf {b}
    b.mkdir(exist_ok=True)

# listeners
zip_button.on_click(zip_outputs)
clean_button.on_click(zip_outputs)
zip_clean_button.on_click(zip_outputs)

    
# show them
display(
    HBox([output_control, zip_button, clean_button,zip_clean_button]),
    persistent_config_control,
    persistence_control,
    force_cache_control,
    extn_update_control,
    hf_token,
    commandline_arg_control,
)

#default links
active_urls={"models":[],"vae":[],"lora":[],"extensions":[]}

# links
links_file = 'links.json'
links={}
persistant_urls={"models":[],"vae":[],"lora":[],"extensions":[]}

def parse_url(url):
    parts = url.split(':')
    # Filter out 'enabled' and 'keep' from the parts
    element_parts = [part for part in parts if part not in ['enabled', 'keep']]
    # Join the remaining parts back into a string
    element = ':'.join(element_parts)
    enabled = 'enabled' in parts
    keep = 'keep' in parts
    return element, enabled, keep

            
def update_links(url, enable, keep):
    url.replace(':enabled:keep', '')
    for cat_key, cat_value in links.items():
        for index, item in enumerate(cat_value):            
            if url == item.replace(':enabled', '').replace(':keep', ''):
                cat_value.pop(index)
                modified_url = url
                if enable:
                    modified_url += ":enabled"
                if keep:
                    modified_url += ":keep"
                cat_value.append(modified_url) 
            
def create_toggles(element, enabled, keep):
    style = 'success' if enabled else 'danger'
    toggle_enable =ToggleButton(
        layout=Layout(width='90px'),
        value=enabled, 
        description="Active",
        tooltip=element,
        button_style=style,
        icon="play",
    )

    style = 'success' if keep else 'danger'
    toggle_keep =ToggleButton(
        layout=Layout(width='90px'),
        value=keep, 
        description="Save",
        tooltip=element,
        button_style=style,
        icon="file",
    )
    
    def on_toggle(change):
        update_links(toggle_enable.tooltip,toggle_enable.value,toggle_keep.value)
        toggle_keep.button_style = 'success' if toggle_keep.value else 'danger'
        toggle_enable.button_style = 'success' if toggle_enable.value else 'danger'
            
    toggle_enable.observe(on_toggle, 'value')
    toggle_keep.observe(on_toggle, 'value')
    return toggle_enable , toggle_keep


if os.path.exists(links_file):
    with open(links_file, 'r') as file:
        links = json.load(file)
    
    for category, urls in links.items():
        if len(urls) >0 :
            print(f"#{category}")
            for url in urls:
                element, enabled, keep = parse_url(url) 

                #todo later
                filename=element
                
                display(
                    HBox([Box(layout=Layout(width='50px'))] +list(create_toggles(element, enabled, keep))+ [Label(value=filename)])
                )


#### 2. Start 🚀

In [ ]:
import os
import zipfile
import time
import shutil
from subprocess import check_call
from pathlib import Path
from tempfile import NamedTemporaryFile
from urllib.parse import urlparse
from os import getenv, environ

# --------------------- Edit these if needed


active_urls["models"] =[    
    "https://huggingface.co/NoCrypt/expermental_models/resolve/main/shux7.safetensors",
]

active_urls["vae"] =[    
    "https://huggingface.co/NoCrypt/resources/resolve/main/VAE/any.vae.safetensors",
    "https://huggingface.co/NoCrypt/resources/resolve/main/VAE/blessed2.vae.safetensors",
    "https://huggingface.co/NoCrypt/resources/resolve/main/VAE/wd-blessed09.vae.safetensors",
    "https://huggingface.co/NoCrypt/resources/resolve/main/VAE/wd.vae.safetensors",
    "https://civitai.com/api/download/models/88156", # clear vae
]

active_urls["lora"] =[    
    # "https://civitai.com/api/download/models/31228", # 3232 pixels
]

active_urls["extensions"] =[    
    #"https://github...",
]

# ---------------------

# options class for easy access
class CurrentOptions:
    # fixed (Always present) command line args, needed for this to work at all
    static_args = (
        "--enable-insecure-extension-access --disable-safe-unpickle --theme dark  --listen --port 6006"
    )

    config_file = ""

    @property
    def persistent_output(self) -> bool:
        return persistence_control.value
        
    @property
    def persistent_config(self) -> bool:
        return persistent_config_control.value
    
    @property
    def force_cache_deps(self) -> bool:
        return force_cache_control.value

    @property
    def update_all_extensions(self) -> bool:
        return extn_update_control.value

    @property
    def optional_huggingface_token(self) -> str:
        return hf_token.value
  
    @property
    def commandline_arguments(self) -> str:
        arg_list = [self.static_args]
        if hf_token.value != "":
            arg_list.extend(["--hf-token-out", hf_token.value])
        if extn_update_control.value is True:
            arg_list.append("--update-all-extensions")
        if persistent_config_control.value is True and self.config_file != "":
            arg_list.extend(["--ui-settings-file", self.config_file])
        if commandline_arg_control.value != "":
            arg_list.extend([x for x in commandline_arg_control.value.split(" ") if x != ""])
        return " ".join(arg_list)


# spawn the boi
options = CurrentOptions()

# paperspace-provided directories
notebook_dir = Path("/notebooks")
storage_dir = Path("/storage")

# local-only semi-persistent storage
content_dir = Path("/content")

# symlink the directory for easier access
content_dir.mkdir(exist_ok=True)
content_symlink = notebook_dir.joinpath("content")
if not all(
    (
        content_symlink.is_symlink(),
        content_symlink.resolve() == content_dir,
    )
):
    # recreate the symlink
    content_symlink.unlink(missing_ok=True)
    content_symlink.symlink_to(content_dir, target_is_directory=True)


# where we put deps
dep_cache_dir = storage_dir.joinpath("a1111_dependencies")

# where models will go
sdw_dir = content_dir.joinpath("sdw")
models_dir = sdw_dir.joinpath("models")
extensions_dir = sdw_dir.joinpath("extensions")
output_dir = notebook_dir.joinpath("outputs/") 
models_persistance_dir = notebook_dir.joinpath("models")

# options for aria2 to use
aria2_opts = [ "--console-log-level=error", "--summary-interval=60", "-j1", "-x1", "-s1", "-k1M", "-c"]

# download repo if needed
config_json = content_dir.joinpath("sdw/config.json")
persistent_config_file = notebook_dir.joinpath("config.json")

if not config_json.exists() :
    # repo isn't here so LET'S GOOOOOOOOOO
    aria2_task = "\n".join(
        [
            "https://huggingface.co/NoCrypt/fast-repo/resolve/main/repo.tar.lz4",
            "\tout=repo.tar.lz4",
            "https://huggingface.co/NoCrypt/fast-repo/resolve/main/cache.tar.lz4",
            "\tout=cache.tar.lz4",
        ]
    )

    with NamedTemporaryFile("w") as aria2_file:
        aria2_file.write(aria2_task)
        aria2_file.flush()
        display("Downloading repo and cache. This may take a while...")
        check_call(
            ["aria2c", f"--input-file={aria2_file.name}"] + aria2_opts,
            cwd=content_dir,
        )

    display("Extracting repo...")
    check_call(["tar", "-xI", "lz4", "-f", "repo.tar.lz4", "--directory=/"], cwd=content_dir)
    content_dir.joinpath("repo.tar.lz4").unlink()

    display("Extracting cache...")
    check_call(["tar", "-xI", "lz4", "-f", "cache.tar.lz4", "--directory=/"], cwd=content_dir)
    content_dir.joinpath("cache.tar.lz4").unlink()

if options.persistent_config :
    if not persistent_config_file.exists():
        !cp {config_json} {notebook_dir}
    config_json=persistent_config_file

options.config_file= str(config_json)

# deal with persistent output 
if options.persistent_output:
  
  if not output_dir.exists():
      output_dir.mkdir(exist_ok=True)

  !sed -i 's@"outdir_txt2img_samples": "outputs/txt2img-images"@"outdir_txt2img_samples": "{output_dir}/txt2img-images"@' {config_json}
  !sed -i 's@"outdir_img2img_samples": "outputs/img2img-images"@"outdir_img2img_samples": "{output_dir}/img2img-images"@' {config_json}
  !sed -i 's@"outdir_extras_samples": "outputs/extras-images"@"outdir_extras_samples": "{output_dir}/extras-images"@' {config_json}
  !sed -i 's@"outdir_txt2img_grids": "outputs/txt2img-grids"@"outdir_txt2img_grids": "{output_dir}/txt2img-grids"@' {config_json}
  !sed -i 's@"outdir_img2img_grids": "outputs/img2img-grids"@"outdir_img2img_grids": "{output_dir}/img2img-grids"@' {config_json}
  !sed -i 's@"outdir_save": "log/images"@"outdir_save": "{output_dir}/log/images"@' {config_json}

# set up dependency cache
if dep_cache_dir.exists() is False or options.force_cache_deps:
    if options.force_cache_deps and dep_cache_dir.exists():
        display("Forcing removal of dependency cache!")
        shutil.rmtree(dep_cache_dir)
    display("Caching dependencies. This may take a while...")
    dep_cache_dir.mkdir(exist_ok=True)
    check_call(
        ["bash", "./webui.sh", "-f", options.commandline_arguments, "--exit"],
        cwd=content_dir.joinpath("sdw"),
    )

    # recursively copy all files in /usr/local/lib/python3.10/site-packages/ to /storage/a1111_dependencies/ (except for .pyc files)
    if Path("/usr/local/lib/python3.10/site-packages/").exists():
        display("Copying dependencies to persistent cache...")
        shutil.copytree(
            "/usr/local/lib/python3.10/site-packages/",
            "/storage/a1111_dependencies/",
            symlinks=True,
            ignore=shutil.ignore_patterns("*.pyc"),
            dirs_exist_ok=True,
        )

# func to acquire checkpoints with
def acquire_checkpoints(checkpoint_urls: list[str], fldr: str = "Stable-diffusion", keep: bool = False):
    dst=models_dir.joinpath(fldr)
    real_dst=models_dir.joinpath(fldr)
    if(keep) :
        if not models_persistance_dir.exists():
            models_persistance_dir.mkdir(exist_ok=True)
        dst=models_persistance_dir.joinpath(fldr)
        if not dst.exists():
            dst.mkdir(exist_ok=True)
   
    aria2_task_lines = []
    for url in checkpoint_urls:
        if 'drive.google' in url:
            if 'folders' in url:
                !cd {dst} && gdown --folder {url} --fuzzy -c
            else:
                !cd {dst} && gdown {url} --fuzzy -c
        elif 'civitai' in url :
            aria2_task_lines.extend([url,""])
        else :
            filename = urlparse(url).path.split("/")[-1]
            if not dst.joinpath(filename).exists():
                aria2_task_lines.extend([url, f"\tout={filename}"])
    if len(aria2_task_lines) > 0:
        with NamedTemporaryFile("w") as aria2_file:
            aria2_file.write("\n".join(aria2_task_lines))
            aria2_file.flush()
            display(f"Downloading {len(aria2_task_lines)} checkpoints, this may take a while...")
            check_call(
                ["aria2c", f"--input-file={aria2_file.name}"] + aria2_opts,
                cwd=dst,
            )
        if keep:
            display("Setting up links...")
            for file in dst.iterdir():
                original_file = real_dst / file.name
                symlink_path = file
                if original_file.exists() or original_file.is_symlink():
                    print(f"Removing existing file or symlink: {original_file}")
                    original_file.unlink()
                try:
                    original_file.symlink_to(symlink_path)
                    print(f"Created symbolic link: {original_file} -> {symlink_path}")
                except FileExistsError as e:
                    print(f"Failed to create symbolic link due to existing file: {e}")
        display("Done")
    else:
        display("All checkpoints already downloaded!")

# func to acquire extensions 
def acquire_extensions(extensions_urls: list[str]):
    for url in extensions_urls:
        repo_name = url.split('/')[-1]
        if extensions_dir.joinpath(repo_name).exists():
            !cd {extensions_dir}/{repo_name} && git fetch && git merge
        else:
            !git clone {url} {extensions_dir}/{repo_name}
    display("All extensions acquired!")


if links :
    display("Saving links ...")
    with open(links_file, 'w') as file:
        json.dump(links, file, indent=4)
    display("Loading links ...")
    for category, urls in links.items():
        if len(urls) >0 :
            for url in urls:
                element, enabled, keep = parse_url(url)
                if enabled :
                    if not keep :
                        active_urls[category].append(element)
                        if element in persistant_urls[category]:
                            persistant_urls[category].remove(element)
                    else :
                        persistant_urls[category].append(element)
                        if element in active_urls[category]:
                            active_urls[category].remove(element)
    acquire_checkpoints(persistant_urls["models"], "Stable-diffusion", True)    
    acquire_checkpoints(persistant_urls["vae"], "VAE", True)    
    acquire_checkpoints(persistant_urls["lora"], "Lora", True)
    

# download the rest to content
acquire_checkpoints(active_urls["models"])
acquire_checkpoints(active_urls["vae"], "VAE")
acquire_checkpoints(active_urls["lora"],"Lora")
acquire_extensions(active_urls["extensions"])

# start. the thing
environ["COMMANDLINE_ARGS"] = options.commandline_arguments
environ["REQS_FILE"] = "requirements_versions.txt"

display("Starting 'tensorboard'...")
display(f"Access via this URL: https://tensorboard-{getenv('PAPERSPACE_FQDN')}")

webui_user = f"""#!/bin/bash
export COMMANDLINE_ARGS='{options.commandline_arguments}'
venv_dir='-'
"""
content_dir.joinpath("sdw/webui-user.sh").write_text(webui_user)
check_call(["/usr/bin/env", "bash", "./webui.sh", "-f"], cwd=content_dir.joinpath("sdw"))


In [ ]:
# code to update webui
!cd {sdw_dir} && git pull